In [1]:
import os
import torch
import time
from PIL import Image
import numpy as np
from torchvision import transforms

In [2]:
#checking the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
#importing the models from the compressai zoo
from compressai.zoo import models
#models avaialble are 
print("models are:",models.keys())

quality=4
metric="ms-ssim" #"mse"
#model_name="bmshj2018-factorized"
#model_name="mbt2018-mean"
#model_name="mbt2018-mean"
model_name="cheng2020-anchor"

net = models[model_name](quality=quality,metric=metric, pretrained=True).eval().to(device)
print("Model encoder architecture",model_name)

print(f'Encoder Parameters: {sum(p.numel() for p in net.parameters())}')

models are: dict_keys(['bmshj2018-factorized', 'bmshj2018-factorized-relu', 'bmshj2018-hyperprior', 'mbt2018-mean', 'mbt2018', 'cheng2020-anchor', 'cheng2020-attn', 'bmshj2018-hyperprior-vbr', 'mbt2018-mean-vbr', 'mbt2018-vbr', 'hrtzxf2022-pcc-rec', 'sfu2023-pcc-rec-pointnet', 'sfu2024-pcc-rec-pointnet2-ssg', 'ssf2020'])
Model encoder architecture cheng2020-anchor
Encoder Parameters: 11833149


In [5]:
#trying the PyTorch exporting of the model
path_to_model_export=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\PyTorch_inference\pytorch_models\cheng2020_anchor.pth"
#torch.save(net,path_to_model_export)
#print("model is exported")

#trying to load the model
model_imported=torch.load(path_to_model_export,weights_only=False)
model_imported.eval()

Cheng2020Anchor(
  (entropy_bottleneck): EntropyBottleneck(
    (likelihood_lower_bound): LowerBound()
    (matrices): ParameterList(
        (0): Parameter containing: [torch.float32 of size 128x3x1 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 128x3x3 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 128x3x3 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 128x3x3 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 128x1x3 (cuda:0)]
    )
    (biases): ParameterList(
        (0): Parameter containing: [torch.float32 of size 128x3x1 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 128x3x1 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 128x3x1 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 128x3x1 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 128x1x1 (cuda:0)]
    )
    (factors): ParameterList(
        (0): Parameter containing: 

In [4]:
#details of the model architecture
# Print model structure and input type
print("Model encoder architecture:", model_name)
print(net)  # Print the model structure
#print(f"Expected input shape: {net.input_shape}")
#print(f'Encoder Parameters: {sum(p.numel() for p in net.parameters())}')

Model encoder architecture: cheng2020-anchor
Cheng2020Anchor(
  (entropy_bottleneck): EntropyBottleneck(
    (likelihood_lower_bound): LowerBound()
    (matrices): ParameterList(
        (0): Parameter containing: [torch.float32 of size 128x3x1 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 128x3x3 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 128x3x3 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 128x3x3 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 128x1x3 (cuda:0)]
    )
    (biases): ParameterList(
        (0): Parameter containing: [torch.float32 of size 128x3x1 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 128x3x1 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 128x3x1 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 128x3x1 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 128x1x1 (cuda:0)]
    )
    (factors): Param

In [5]:
#random tensor of the size for the testing
numpy_tensor = np.random.randn(1280, 1280,3).astype(np.float32)
x = transforms.ToTensor()(numpy_tensor)
x=x.unsqueeze(0).to(device)
print("type:", type(x), "shape:", x.shape)


type: <class 'torch.Tensor'> shape: torch.Size([1, 3, 1280, 1280])


In [6]:
#image processing for the model 
path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\image.png"
image=Image.open(path)
#image=image.resize((2420,2420)) #this resolution also works
image=image.resize((1280,1280)) #this resolution also works inference time 0.36 seconds for the ssim,8,mbt2018

#trying the normalization step
#image=np.array(image)/255.0
#image=image.astype(np.float32)

image=transforms.ToTensor()(image).unsqueeze(0).to(device)
print("final image shape:",image.shape)

final image shape: torch.Size([1, 3, 1280, 1280])


In [248]:
#single channel gray version testing 
#load the image as gray and resize and gray
image_=Image.open(path).convert("L")
image_=image.resize((1280,1280))
#make the numpy zeros tensor of the size
np_gray=np.zeros((3,1280,1280))
#assign the data to the first axis of the gray
np_gray[0]=np.array(image_)
np_gray=np_gray.transpose().astype(np.float32)
#check the gray matrix
#print(np_gray)
print("gray matrix shape:",np_gray.shape)

image_gray=transforms.ToTensor()(np_gray).unsqueeze(0).to(device)
print(image_gray.shape)
#check the type of the tensor
#print(image_gray)


RuntimeError: requested resize to (1280, 1280) ((1280, 1280) elements in total), but the given tensor has a size of 1x3x1280x1280 (4915200 elements). autograd's resize can only change the shape of a given tensor, while preserving the number of elements. 

mbt2018 model works in the multiples of the 128x128 like 1280 and 1280 but fails in the other configs like the HD (720,1280)

In [9]:
#run the forward pass computation 
start=time.time()
with torch.no_grad():
    #out_net = net.forward(x)
    out_net=net.forward(image)
out_net['x_hat'].clamp_(0, 1)
end=time.time()
print("Compression Time:",round(end-start,2))

output=out_net['x_hat']
print("output type is:",type(output))
rec_net = transforms.ToPILImage()(out_net['x_hat'].squeeze())
output_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\PyTorch_inference\output.jpg"
#rec_net.save(output_path,format="JPEG")
rec_net.show()

Compression Time: 0.45
output type is: <class 'torch.Tensor'>
